# Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

# Setting Image and Batch size

In [1]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

# Importing the train, test, and val data

In [ ]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)


# Preprocessing the data for VGG16

In [ ]:
train = train.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
val = val.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
test = test.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))

# Defining the model

In [ ]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    pooling='avg',
    classes=6,
    classifier_activation="softmax"
)

model.compile(optimizer=keras.optimizers.RMSprop(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Running the Model

In [ ]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath="models/sota/SOTA_VGG16.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history = model.fit(
  train.cache(),
  epochs=20,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

# Evaluating the model

In [ ]:
model.evaluate(test)

load_model = keras.models.load_model("models/sota/SOTA_VGG16.h5")
load_model.evaluate(test)